In [2]:
import os
import math
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import tensorflow as tf
from data_preparation_utils import *
from machine_learning_utils import *

### Load data

In [4]:
# Specify the base folder where all your image classes are stored
folder = 'data'

In [ ]:
start_data = datasets.ImageFolder(folder, transform=transforms.ToTensor())

### Check min and max height and width of the images.

In [8]:
check_max_min_dimensions(start_data)

Min Width: 71, Max Width: 6283
Min Height: 51, Max Height: 7786


### Resizing the images to 256x256

In [3]:
transformations = transforms.Compose([transforms.Resize((256, 256), antialias=True), transforms.ToTensor()])
data = datasets.ImageFolder(root = folder, transform=transformations)

### Check if all images have the same dimensions and the same number of channels.

In [10]:
check_dimensions(data)
check_channels(data)

All images have the same shape: torch.Size([256, 256]).
All images have 3 channels.


### Check number of samples in each class.

In [11]:
count_images_per_class(data)

Class battery: 944 images
Class biological: 983 images
Class cardboard: 1810 images
Class clothes: 5323 images
Class glass: 3039 images
Class metal: 994 images
Class paper: 1650 images
Class plastic: 1915 images
Class shoes: 1977 images
Class trash: 772 images


### Data augmentation to bring all the classes to at least 1500 samples.

In [ ]:
input_dir = "data"
output_dir = "augmented_data"
target_size = 1500

os.makedirs(output_dir, exist_ok=True)

augmentations = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
])


for class_name in os.listdir(input_dir):
    input_class_dir = os.path.join(input_dir, class_name)

    num_images = len([img for img in os.listdir(input_class_dir) if img.endswith(('.jpg', '.jpeg', '.png'))])

    if num_images < target_size:
        augment_images(class_name, input_class_dir, num_images, target_size, augmentations=augmentations)
    else:
        print(f"Class {class_name} already has {num_images} images, no augmentation needed.")

print("Data augmentation completed.")

Augmenting 556 images for class: battery
Augmenting 517 images for class: biological
Class cardboard already has 1810 images, no augmentation needed.
Class clothes already has 5323 images, no augmentation needed.
Class glass already has 3039 images, no augmentation needed.
Augmenting 506 images for class: metal
Class paper already has 1650 images, no augmentation needed.
Class plastic already has 1915 images, no augmentation needed.
Class shoes already has 1977 images, no augmentation needed.
Augmenting 728 images for class: trash
Data augmentation completed.


### Train Test Split

In [ ]:
# Paths
original_dataset_dir = "data"
output_dir = "final dataset"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Create train/test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Parameters
test_size = 0.2 
random_state = 42

# Split the dataset
split_dataset(original_dataset_dir, train_dir, test_dir, test_size=test_size, random_state=random_state)

Class battery -> Train: 1200 | Test: 300
Class biological -> Train: 1200 | Test: 300
Class cardboard -> Train: 1448 | Test: 362
Class clothes -> Train: 4258 | Test: 1065
Class glass -> Train: 2431 | Test: 608
Class metal -> Train: 1200 | Test: 300
Class paper -> Train: 1320 | Test: 330
Class plastic -> Train: 1532 | Test: 383
Class shoes -> Train: 1581 | Test: 396
Class trash -> Train: 1200 | Test: 300
